In [15]:
import nltk

In [2]:
nltk.download("all", quiet=True)

True

In [16]:
from nltk import word_tokenize, pos_tag, ne_chunk
sentence = "Jim bought 300 shares of Acme Corp. in 2006."
sentence = pos_tag(word_tokenize(sentence))
print(sentence)

[('Jim', 'NNP'), ('bought', 'VBD'), ('300', 'CD'), ('shares', 'NNS'), ('of', 'IN'), ('Acme', 'NNP'), ('Corp.', 'NNP'), ('in', 'IN'), ('2006', 'CD'), ('.', '.')]


In [60]:
chunk_sentence = ne_chunk(sentence)
print(chunk_sentence)

(S
  (PERSON Jim/NNP)
  bought/VBD
  300/CD
  shares/NNS
  of/IN
  (ORGANIZATION Acme/NNP Corp./NNP)
  in/IN
  2006/CD
  ./.)


In [74]:
import spacy
from collections import Counter
import en_core_web_sm
nlp = spacy.load("en_core_web_sm")
spacy.prefer_gpu()

True

In [68]:
!pip install en_core_web_sm

ERROR: Could not find a version that satisfies the requirement en_core_web_sm (from versions: none)
ERROR: No matching distribution found for en_core_web_sm


In [86]:
doc = nlp("Jim bought 300 shares of Acme Corp. in 2006.")
for entity in doc.ents:
    print(type(entity.text))
    print(type(entity.label_))
    result.append(entity.text+'/'+entity.label_)
print(result)


<class 'str'>
<class 'str'>


AttributeError: 'tuple' object has no attribute 'append'

In [70]:
print([x.ent_iob_ for x in sentence])

AttributeError: 'str' object has no attribute 'ent_iob_'

In [58]:
for result in chunk_sentence:
    if isinstance(result, tuple) != True:
        print(result)
    # if len(result) == 1:
    #     print(result)
    # else:
    #     print(result)

(PERSON Jim/NNP)
(ORGANIZATION Acme/NNP Corp./NNP)


In [56]:

result = []
for ner_result in chunk_sentence:
    if isinstance(ner_result, tuple) != True:
        print(ner_result)
        for ner in ner_result:
            print(ner)
            result.append([ner_result[0], ner])


(PERSON Jim/NNP)
('Jim', 'NNP')
(ORGANIZATION Acme/NNP Corp./NNP)
('Acme', 'NNP')
('Corp.', 'NNP')


In [47]:
print(result)

[[('Jim', 'NNP'), ('Jim', 'NNP')], [('Acme', 'NNP'), ('Acme', 'NNP')], [('Acme', 'NNP'), ('Corp.', 'NNP')]]


In [11]:
sentence = "acer born in march 1997 and he began to learn nlp_task after 26"
ner_sentence = pos_tag(word_tokenize(sentence))
print(ner_sentence)


[('acer', 'NN'), ('born', 'VBN'), ('in', 'IN'), ('march', 'NN'), ('1997', 'CD'), ('and', 'CC'), ('he', 'PRP'), ('began', 'VBD'), ('to', 'TO'), ('learn', 'VB'), ('nlp_task', 'NN'), ('after', 'IN'), ('26', 'CD')]
